In [15]:
import pandas as pd
import altair as alt
import numpy as np


df = pd.read_csv('Marvel_inversión.csv', sep=';')


df.columns = ['Titulo', 'Anio_Estreno', 'Tipo', 'Inversion', 'Columna_Vacia_1', 'Columna_Vacia_2']


df = df.dropna(subset=['Anio_Estreno', 'Tipo', 'Inversion'])
df = df.iloc[:-1]

try:
    df['Inversion_USD'] = df['Inversion'].astype(float)
except ValueError:

    df['Inversion_USD'] = df['Inversion'].str.replace('.', '', regex=False).astype(float)

df['Anio_Estreno'] = df['Anio_Estreno'].astype(int)



df['Tipo_Limpio'] = df['Tipo'].str.strip()


df['Inversion_MM'] = df['Inversion_USD'] / 1_000_000


df['Categoria'] = df['Tipo_Limpio'].replace({
    'Secuela/Precuela': 'Secuela/Precuela',
    'Secuela': 'Secuela/Precuela',
    'Precuela': 'Secuela/Precuela',
    'Original': 'Original'
})


color_scale = alt.Scale(
    domain=['Original', 'Secuela/Precuela'],
    range=['#1c5791', '#D41C16']
)


tooltip_data = [
    alt.Tooltip('Titulo', title='Película'),
    alt.Tooltip('Anio_Estreno', title='Año de Estreno'),
    alt.Tooltip('Inversion_USD', title='Inversión (USD)', format='$,.0f')
]




y_max_data = df['Inversion_MM'].max()

chart = alt.Chart(df).mark_circle(size=100).encode(

    x=alt.X('Anio_Estreno:Q', title='Año de Estreno',
            scale=alt.Scale(domain=[df['Anio_Estreno'].min() - 0.5, df['Anio_Estreno'].max() + 0.5])),


    y=alt.Y('Inversion_MM:Q', title='Inversión (Millones de USD)',
            scale=alt.Scale(domain=[100.0, y_max_data * 1.05])),


    color=alt.Color('Categoria:N', scale=color_scale, title='Tipo de Película'),


    tooltip=tooltip_data
).properties(
    title=' Inversión en Películas Marvel por Año y Tipo (Altair)'
).interactive()

chart.display()

alt.Chart(...)